# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mysql.connector

from sklearn.linear_model import LassoLarsCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler, StandardScaler

In [2]:
import mysql.connector
import pandas as pd

def get_sql_result(SQL: str) -> pd.DataFrame:
    try:
        conn = mysql.connector.connect(
            user='chad',
            password='buggersZoologyBrothCoda',
            host="data-engineering-cluster.tfs.vision",
            database="test",
            port=30930,
        )
        cursor = conn.cursor(dictionary=True)
        
        cursor.execute(SQL)
        df = pd.DataFrame(cursor.fetchall())
        
        cursor.close()
        conn.close()
        
        return df
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        if 'conn' in locals() and conn.is_connected():
            conn.close()
        return pd.DataFrame()  # Return an empty DataFrame in case of error


In [3]:
test_df = get_sql_result('SELECT * FROM electricity_energy_power LIMIT 100')

Error: 2003 (HY000): Can't connect to MySQL server on 'data-engineering-cluster.tfs.vision:30930' (111)


# Dataframe

## Solar df

### VCOM data

In [4]:
file_path = 'Mvusuludzo VCOM.csv'

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(file_path)
#test_df = get_sql_result('SELECT * FROM solcast_data LIMIT 100')

# Select only relevant columns from .csv
selected_columns = ['tstamp','ptot']
new_df = df[selected_columns]

In [5]:
new_df['ptot'] = new_df['ptot'].abs()

# Convert the object column to datetime
new_df['tstamp'] = pd.to_datetime(new_df['tstamp'])

# Set the timestamp column as the DataFrame index
new_df.set_index('tstamp', inplace=True)

In [6]:
new_df

,ptot
tstamp,
2023-06-01 00:00:00,NaN
2023-06-01 00:05:00,NaN
2023-06-01 00:10:00,NaN
2023-06-01 00:15:00,NaN
2023-06-01 00:20:00,NaN
...,...
2024-05-31 23:35:00,0.0
2024-05-31 23:40:00,0.0
2024-05-31 23:45:00,0.0


In [7]:
# Define the specific timestamp range
start_timestamp = pd.Timestamp('2023-06-01')  # Example start timestamp
end_timestamp = pd.Timestamp('2024-05-31')    # Example end timestamp

# Filter the DataFrame based on the timestamp range
solar_df = new_df[(new_df.index >= start_timestamp) & (new_df.index <= end_timestamp)]

solar_df

,ptot
tstamp,
2023-06-01 00:00:00,NaN
2023-06-01 00:05:00,NaN
2023-06-01 00:10:00,NaN
2023-06-01 00:15:00,NaN
2023-06-01 00:20:00,NaN
...,...
2024-05-30 23:40:00,0.0
2024-05-30 23:45:00,0.0
2024-05-30 23:50:00,0.0


In [8]:
# Define the start and end timestamps for the range
start_timestamp = pd.Timestamp('2024-03-08 15:00:00')
end_timestamp = pd.Timestamp('2024-03-08 23:55:55')

# Filter rows based on the timestamp range
filtered_solar_df = solar_df[(solar_df.index >= start_timestamp) & (solar_df.index < end_timestamp)]

filtered_solar_df.head(60)

,ptot
tstamp,
2024-03-08 15:00:00,26.821
2024-03-08 15:05:00,26.138
2024-03-08 15:10:00,25.399
2024-03-08 15:15:00,24.755
2024-03-08 15:20:00,24.051
2024-03-08 15:25:00,23.290
2024-03-08 15:30:00,22.637
2024-03-08 15:35:00,21.911
2024-03-08 15:40:00,21.157


In [9]:
solar_df_new = []

# Resample with a frequency of 5 minute and fill gaps with 0
solar_df_resampled = solar_df.resample('5T').sum().fillna(0)

solar_df_new.append(solar_df_resampled)

df_f = pd.concat(solar_df_new)

df_f

,ptot
tstamp,
2023-06-01 00:00:00,0.0
2023-06-01 00:05:00,0.0
2023-06-01 00:10:00,0.0
2023-06-01 00:15:00,0.0
2023-06-01 00:20:00,0.0
...,...
2024-05-30 23:40:00,0.0
2024-05-30 23:45:00,0.0
2024-05-30 23:50:00,0.0


### Copper data

In [10]:
file_path = 'Mvusuludzo PV Meter DG.csv'

# Read the CSV file into a Pandas DataFrame
cop_df = pd.read_csv(file_path)

# Select only relevant columns from .csv
selected_columns = ['timestamp', 'export_kwh', 'ptot']
new_cop_df = cop_df[selected_columns]

# Make ptot values positive
new_cop_df['ptot'] = new_cop_df['ptot'].abs()

# Convert the object column to datetime
new_cop_df['timestamp'] = pd.to_datetime(new_cop_df['timestamp'])

# Shift the tstamp column by 2 hours
new_cop_df['timestamp'] = new_cop_df['timestamp'] - pd.DateOffset(hours=-2)

# Set the timestamp column as the DataFrame index
new_cop_df.set_index('timestamp', inplace=True)

# Rename index 'tstamp'
new_cop_df.rename_axis('tstamp', inplace=True)

# Rename ptot column copper_ptot
new_cop_df.rename(columns={'ptot': 'copper_ptot'}, inplace=True)

/tmp/ipykernel_99594/2920311161.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_cop_df['ptot'] = new_cop_df['ptot'].abs()
/tmp/ipykernel_99594/2920311161.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_cop_df['timestamp'] = pd.to_datetime(new_cop_df['timestamp'])
/tmp/ipykernel_99594/2920311161.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [11]:
new_cop_df

,export_kwh,copper_ptot
tstamp,,
2023-05-01 02:00:00,0.0,0.0
2023-05-01 02:05:00,0.0,0.0
2023-05-01 02:10:00,0.0,0.0
2023-05-01 02:15:00,0.0,0.0
2023-05-01 02:20:00,0.0,0.0
...,...,...
2024-06-30 23:40:00,0.0,0.0
2024-06-30 23:45:00,0.0,0.0
2024-06-30 23:50:00,0.0,0.0


In [12]:
# Define the specific timestamp range
start_timestamp = pd.Timestamp('2023-06-01')  # Example start timestamp
end_timestamp = pd.Timestamp('2024-05-31')    # Example end timestamp

# Filter the DataFrame based on the timestamp range
gap_df = new_cop_df[(new_cop_df.index >= start_timestamp) & (new_cop_df.index <= end_timestamp)]

gap_df

,export_kwh,copper_ptot
tstamp,,
2023-06-01 01:10:00,0.0,0.0
2023-06-01 01:15:00,0.0,0.0
2023-06-01 01:20:00,0.0,0.0
2023-06-01 01:25:00,0.0,0.0
2023-06-01 01:30:00,0.0,0.0
...,...,...
2024-05-30 23:40:00,0.0,0.0
2024-05-30 23:45:00,0.0,0.0
2024-05-30 23:50:00,0.0,0.0


In [13]:
# Function to clean and convert the 'export_wh' column
#def clean_export_wh(value):
    # Replace ',' with '' and convert to float
#    return float(value.replace(',', '').replace(' ', ''))

# Apply the cleaning function to 'export_wh'
#gap_df['export_kwh'] = gap_df['export_kwh'].apply(clean_export_wh)

# Resample with a frequency of 5 minutes and sum values
gap_df_resampled = gap_df.resample('5T').sum()

# Fill gaps in 'export_wh' column with 0
gap_df_resampled['export_kwh'] = gap_df_resampled['export_kwh'].fillna(0)
gap_df_resampled['copper_ptot'] = gap_df_resampled['copper_ptot'].fillna(0)

gap_df_new = [gap_df_resampled]

cop_df_f = pd.concat(gap_df_new)

# Divide each value in the 'data' column by 10
cop_df_f['copper_ptot'] = cop_df_f['copper_ptot'] / 10

cop_df_f

,export_kwh,copper_ptot
tstamp,,
2023-06-01 01:10:00,0.0,0.0
2023-06-01 01:15:00,0.0,0.0
2023-06-01 01:20:00,0.0,0.0
2023-06-01 01:25:00,0.0,0.0
2023-06-01 01:30:00,0.0,0.0
...,...,...
2024-05-30 23:40:00,0.0,0.0
2024-05-30 23:45:00,0.0,0.0
2024-05-30 23:50:00,0.0,0.0


### Combine VCOM and Copper DF

In [14]:
# Merge on the index column 'tstamp'
solar_df = pd.merge(df_f, cop_df_f, left_index=True, right_index=True)

solar_df

,ptot,export_kwh,copper_ptot
tstamp,,,
2023-06-01 01:10:00,0.0,0.0,0.0
2023-06-01 01:15:00,0.0,0.0,0.0
2023-06-01 01:20:00,0.0,0.0,0.0
2023-06-01 01:25:00,0.0,0.0,0.0
2023-06-01 01:30:00,0.0,0.0,0.0
...,...,...,...
2024-05-30 23:40:00,0.0,0.0,0.0
2024-05-30 23:45:00,0.0,0.0,0.0
2024-05-30 23:50:00,0.0,0.0,0.0


In [15]:
# Define the start and end timestamps for the range
start_timestamp = pd.Timestamp('2024-03-05 14:15:00')
end_timestamp = pd.Timestamp('2024-03-05 19:00:00')

# Filter rows based on the timestamp range
df_f_filtered = solar_df[(solar_df.index >= start_timestamp) & (solar_df.index < end_timestamp)]

df_f_filtered.head(60)

,ptot,export_kwh,copper_ptot
tstamp,,,
2024-03-05 14:15:00,32.164,32.26,38.7120
2024-03-05 14:20:00,31.623,31.81,38.0452
2024-03-05 14:25:00,31.151,31.10,37.4448
2024-03-05 14:30:00,30.723,30.91,37.0920
2024-03-05 14:35:00,30.143,30.21,36.2520
2024-03-05 14:40:00,29.474,29.57,35.4840
2024-03-05 14:45:00,28.884,28.93,34.7160
2024-03-05 14:50:00,28.164,28.22,33.8640
2024-03-05 14:55:00,27.710,27.78,33.3360


## Satellite df

In [16]:
weather_path = 'Mvusuludzo Solcast 30 min.csv'
weather_df = pd.read_csv(weather_path)

weather_df = pd.DataFrame(weather_df)
weather_df['PeriodEnd'] = pd.to_datetime(weather_df['PeriodEnd']).dt.tz_convert(None)

weather_df.rename(columns={'PeriodEnd': 'tstamp'}, inplace=True)

# Convert tstamp to datetime if it's not already
weather_df['tstamp'] = pd.to_datetime(weather_df['tstamp'])

# Shift the tstamp column by 2 hours
weather_df['tstamp'] = weather_df['tstamp'] - pd.DateOffset(hours=-2)

In [17]:
weather_df

,tstamp,PeriodStart,Period,AirTemp,Azimuth,CloudOpacity,DewpointTemp,Dhi,Dni,Ebh,Ghi,PrecipitableWater,RelativeHumidity,SurfacePressure,WindDirection10m,WindSpeed10m,Zenith,AlbedoDaily
0,2023-05-31 02:30:00,2023-05-31T00:00:00Z,PT30M,13.5,-85,0.0,12.5,0,0,0,0,25.3,93.5,945.6,14,1.9,148,0.15
1,2023-05-31 03:00:00,2023-05-31T00:30:00Z,PT30M,13.4,-83,6.0,12.4,0,0,0,0,25.8,93.2,945.5,14,1.8,141,0.15
2,2023-05-31 03:30:00,2023-05-31T01:00:00Z,PT30M,13.4,-82,6.0,12.3,0,0,0,0,26.2,93.5,945.3,12,1.7,134,0.15
3,2023-05-31 04:00:00,2023-05-31T01:30:00Z,PT30M,13.2,-80,6.0,12.4,0,0,0,0,26.5,94.4,945.1,11,1.7,127,0.15
4,2023-05-31 04:30:00,2023-05-31T02:00:00Z,PT30M,13.2,-78,9.0,12.4,0,0,0,0,26.7,95.2,945.0,8,1.7,120,0.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17707,2024-06-03 00:00:00,2024-06-02T21:30:00Z,PT30M,16.5,121,0.0,11.6,0,0,0,0,24.9,73.3,941.1,84,2.8,177,0.15
17708,2024-06-03 00:30:00,2024-06-02T22:00:00Z,PT30M,16.2,-102,0.0,11.6,0,0,0,0,25.2,74.4,940.9,87,2.8,176,0.15
17709,2024-06-03 01:00:00,2024-06-02T22:30:00Z,PT30M,15.9,-92,3.0,11.7,0,0,0,0,25.3,75.9,940.7,93,2.7,169,0.15
17710,2024-06-03 01:30:00,2024-06-02T23:00:00Z,PT30M,15.7,-89,0.0,11.7,0,0,0,0,25.2,76.9,940.6,100,2.6,162,0.15


In [18]:
# Define the specific timestamp range
start_timestamp = pd.Timestamp('2023-06-01')  # Example start timestamp
end_timestamp = pd.Timestamp('2024-05-31')    # Example end timestamp

# Filter the DataFrame based on the timestamp range
satellite_df = weather_df[(weather_df['tstamp'] >= start_timestamp) & (weather_df['tstamp'] <= end_timestamp)]

# Set 'tstamp' column as the index
satellite_df.index = pd.to_datetime(satellite_df['tstamp'])

# Remove 'tstamp', 'PeriodStart', and 'Period' columns in one line
satellite_df = satellite_df.drop(['tstamp', 'PeriodStart', 'Period'], axis=1)

satellite_df

,AirTemp,Azimuth,CloudOpacity,DewpointTemp,Dhi,Dni,Ebh,Ghi,PrecipitableWater,RelativeHumidity,SurfacePressure,WindDirection10m,WindSpeed10m,Zenith,AlbedoDaily
tstamp,,,,,,,,,,,,,,,
2023-06-01 00:00:00,17.2,129,43.5,14.8,0,0,0,0,29.5,85.5,943.9,24,1.6,177,0.15
2023-06-01 00:30:00,17.0,-107,43.5,14.7,0,0,0,0,29.3,85.9,943.8,12,1.6,175,0.15
2023-06-01 01:00:00,16.8,-94,41.4,14.6,0,0,0,0,29.1,87.3,943.6,6,1.6,168,0.15
2023-06-01 01:30:00,16.2,-90,37.7,14.4,0,0,0,0,28.7,88.8,943.5,358,1.5,162,0.15
2023-06-01 02:00:00,15.4,-87,24.2,13.8,0,0,0,0,28.3,90.5,943.3,345,1.3,155,0.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-30 22:00:00,12.8,86,0.0,8.2,0,0,0,0,11.2,73.1,944.5,309,1.7,150,0.15
2024-05-30 22:30:00,12.4,88,0.0,7.9,0,0,0,0,10.9,74.0,944.5,304,1.7,157,0.15
2024-05-30 23:00:00,12.1,91,0.0,7.8,0,0,0,0,10.6,75.0,944.4,302,1.7,164,0.15


In [19]:
# Resample DF to 5-minute intervals
satellite_df_resampled = satellite_df.resample('5T').mean()

# Function to handle circular interpolation
def circular_interpolation(series):
    radians = np.deg2rad(series)
    sin_interp = pd.Series(np.sin(radians)).interpolate()
    cos_interp = pd.Series(np.cos(radians)).interpolate()
    return np.rad2deg(np.arctan2(sin_interp, cos_interp))

# Circular interpolation for features measured in degrees
circular_vars = ['Azimuth', 'WindDirection10m', 'Zenith']

# Fill in resampled DF by looping through every row of each feature/column and applying interpolation function
for var in circular_vars:
    satellite_df_resampled[var] = circular_interpolation(satellite_df_resampled[var])

# Linear interpolation on most features
linear_vars = [
    'AirTemp', 'CloudOpacity', 'DewpointTemp', 'Dhi', 
    'Dni', 'Ebh', 'Ghi', 'PrecipitableWater', 
    'RelativeHumidity', 'SurfacePressure', 'WindSpeed10m'
]

# Fill in resampled DF with interpolated values by looping through DF with each feature/column
for var in linear_vars:
    satellite_df_resampled[var] = satellite_df_resampled[var].interpolate(method='linear')
    
# Verify the DF
satellite_df_resampled.head()

,AirTemp,Azimuth,CloudOpacity,DewpointTemp,Dhi,Dni,Ebh,Ghi,PrecipitableWater,RelativeHumidity,SurfacePressure,WindDirection10m,WindSpeed10m,Zenith,AlbedoDaily
tstamp,,,,,,,,,,,,,,,
2023-06-01 00:00:00,17.200000,129.000000,43.5,14.800000,0.0,0.0,0.0,0.0,29.500000,85.500000,943.900000,24.000000,1.6,177.000000,0.15
2023-06-01 00:05:00,17.166667,139.574607,43.5,14.783333,0.0,0.0,0.0,0.0,29.466667,85.566667,943.883333,22.008135,1.6,176.666704,NaN
2023-06-01 00:10:00,17.133333,158.916038,43.5,14.766667,0.0,0.0,0.0,0.0,29.433333,85.633333,943.866667,20.006522,1.6,176.333363,NaN
2023-06-01 00:15:00,17.100000,-169.000000,43.5,14.750000,0.0,0.0,0.0,0.0,29.400000,85.700000,943.850000,18.000000,1.6,176.000000,NaN
2023-06-01 00:20:00,17.066667,-136.916038,43.5,14.733333,0.0,0.0,0.0,0.0,29.366667,85.766667,943.833333,15.993478,1.6,175.666637,NaN


## Format dataframe

### Combine the solar and satellite daytime dataframes and filter nighttime rows

In [20]:
# Merge on the index column 'tstamp'
final_df = pd.merge(satellite_df_resampled, solar_df, left_index=True, right_index=True)

final_df

,AirTemp,Azimuth,CloudOpacity,DewpointTemp,Dhi,Dni,Ebh,Ghi,PrecipitableWater,RelativeHumidity,SurfacePressure,WindDirection10m,WindSpeed10m,Zenith,AlbedoDaily,ptot,export_kwh,copper_ptot
tstamp,,,,,,,,,,,,,,,,,,
2023-06-01 01:10:00,16.60,-92.666907,40.166667,14.533333,0.0,0.0,0.0,0.0,28.966667,87.80,943.566667,3.335262,1.566667,166.000813,NaN,0.0,0.0,0.0
2023-06-01 01:15:00,16.50,-92.000000,39.550000,14.500000,0.0,0.0,0.0,0.0,28.900000,88.05,943.550000,2.000000,1.550000,165.000000,NaN,0.0,0.0,0.0
2023-06-01 01:20:00,16.40,-91.333093,38.933333,14.466667,0.0,0.0,0.0,0.0,28.833333,88.30,943.533333,0.664738,1.533333,163.999187,NaN,0.0,0.0,0.0
2023-06-01 01:25:00,16.30,-90.666366,38.316667,14.433333,0.0,0.0,0.0,0.0,28.766667,88.55,943.516667,-0.669075,1.516667,162.998984,NaN,0.0,0.0,0.0
2023-06-01 01:30:00,16.20,-90.000000,37.700000,14.400000,0.0,0.0,0.0,0.0,28.700000,88.80,943.500000,-2.000000,1.500000,162.000000,0.15,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-30 23:40:00,11.70,106.861017,0.000000,7.666667,0.0,0.0,0.0,0.0,10.333333,76.50,944.366667,-60.666637,1.666667,172.999187,NaN,0.0,0.0,0.0
2024-05-30 23:45:00,11.65,112.500000,0.000000,7.650000,0.0,0.0,0.0,0.0,10.300000,76.70,944.350000,-61.000000,1.650000,174.000000,NaN,0.0,0.0,0.0
2024-05-30 23:50:00,11.60,118.138983,0.000000,7.633333,0.0,0.0,0.0,0.0,10.266667,76.90,944.333333,-61.333363,1.633333,175.000813,NaN,0.0,0.0,0.0


In [21]:
import pandas as pd

# List of columns to check for zero values
columns_to_check = ['Dhi', 'Dni', 'Ebh', 'Ghi', 'ptot']

# Filter the DataFrame to keep rows where at least one of the specified columns is not zero
filtered_df = final_df[(final_df[columns_to_check] != 0).any(axis=1)]

# Filter the DataFrame to remove rows where Ghi < 250 in the morning (00:00 to 12:00)
#morning_filter = (filtered_df.index.hour < 12) & (filtered_df['Ghi'] < 250)
#filtered_df = filtered_df[~morning_filter]

print("DataFrame after removing rows where Dhi, Dni, Ebh, Ghi, and ptot are all zero:")
filtered_df.head(60)

DataFrame after removing rows where Dhi, Dni, Ebh, Ghi, and ptot are all zero:


,AirTemp,Azimuth,CloudOpacity,DewpointTemp,Dhi,Dni,Ebh,Ghi,PrecipitableWater,RelativeHumidity,SurfacePressure,WindDirection10m,WindSpeed10m,Zenith,AlbedoDaily,ptot,export_kwh,copper_ptot
tstamp,,,,,,,,,,,,,,,,,,
2023-06-01 06:35:00,14.416667,-67.500127,18.666667,13.216667,2.166667,10.833333,1.000000,3.166667,22.816667,92.750000,943.166667,-115.666704,0.883333,93.001016,NaN,0.000,0.000,0.0000
2023-06-01 06:40:00,14.433333,-67.000102,16.833333,13.233333,4.333333,21.666667,2.000000,6.333333,22.733333,92.800000,943.233333,-115.333363,0.866667,92.000813,NaN,0.000,0.000,0.0000
2023-06-01 06:45:00,14.450000,-66.500000,15.000000,13.250000,6.500000,32.500000,3.000000,9.500000,22.650000,92.850000,943.300000,-115.000000,0.850000,91.000000,NaN,0.000,0.000,0.0000
2023-06-01 06:50:00,14.466667,-65.999898,13.166667,13.266667,8.666667,43.333333,4.000000,12.666667,22.566667,92.900000,943.366667,-114.666637,0.833333,89.999187,NaN,0.000,0.000,0.0000
2023-06-01 06:55:00,14.483333,-65.499873,11.333333,13.283333,10.833333,54.166667,5.000000,15.833333,22.483333,92.950000,943.433333,-114.333296,0.816667,88.998984,NaN,0.000,0.000,0.0000
2023-06-01 07:00:00,14.500000,-65.000000,9.500000,13.300000,13.000000,65.000000,6.000000,19.000000,22.400000,93.000000,943.500000,-114.000000,0.800000,88.000000,0.15,0.000,0.000,0.0000
2023-06-01 07:05:00,14.583333,-64.500127,8.816667,13.350000,19.166667,98.000000,11.166667,30.333333,22.350000,92.750000,943.566667,-113.833338,0.800000,87.001016,NaN,0.000,0.000,0.0000
2023-06-01 07:10:00,14.666667,-64.000102,8.133333,13.400000,25.333333,131.000000,16.333333,41.666667,22.300000,92.500000,943.633333,-113.666670,0.800000,86.000813,NaN,0.000,0.000,0.0000
2023-06-01 07:15:00,14.750000,-63.500000,7.450000,13.450000,31.500000,164.000000,21.500000,53.000000,22.250000,92.250000,943.700000,-113.500000,0.800000,85.000000,NaN,2.142,2.224,2.6777


### Create a label column for Power Loss Events

In [22]:
def classify_power_events(df):
    power_loss_values = [0] * len(df)
    consecutive_zeros = 0
    last_non_zero_export = None  # Track the last non-zero export_wh value

    for i in range(len(df)):
        export_kwh_value = df.iloc[i]['export_kwh']
        ptot_value = df.iloc[i]['ptot']

        if export_kwh_value == 0:
            consecutive_zeros += 1
            if last_non_zero_export is None:
                last_non_zero_export = export_kwh_value
        else:
            if consecutive_zeros > 0:
                time_in_minutes = consecutive_zeros * 5
                average_increase_per_minute = export_kwh_value/time_in_minutes

                # Check if it's a data gap or a power loss
                if average_increase_per_minute > 1:
                    # It's a data gap
                    power_loss_values[i - consecutive_zeros:i] = [3] * consecutive_zeros
                else:
                    # It's a power loss, determine the type
                    if consecutive_zeros < 22 or consecutive_zeros > 54:
                        power_loss_values[i - consecutive_zeros:i] = [2] * consecutive_zeros
                    else:
                        power_loss_values[i - consecutive_zeros:i] = [1] * consecutive_zeros

            # Reset for next sequence
            consecutive_zeros = 0
            last_non_zero_export = export_kwh_value

            # Mark the current value as normal operation
            power_loss_values[i] = 0

    # Check if the data ends with a zero sequence
    if consecutive_zeros > 0:
        if last_non_zero_export is not None and export_kwh_value > last_non_zero_export:
            power_loss_values[-consecutive_zeros:] = [3] * consecutive_zeros
        else:
            if consecutive_zeros < 22 or consecutive_zeros > 54:
                power_loss_values[-consecutive_zeros:] = [2] * consecutive_zeros
            else:
                power_loss_values[-consecutive_zeros:] = [1] * consecutive_zeros

    return power_loss_values

filtered_df['Power Loss Event'] = classify_power_events(filtered_df)

/tmp/ipykernel_99594/144523917.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Power Loss Event'] = classify_power_events(filtered_df)


In [23]:
filtered_df.head(60)

,AirTemp,Azimuth,CloudOpacity,DewpointTemp,Dhi,Dni,Ebh,Ghi,PrecipitableWater,RelativeHumidity,SurfacePressure,WindDirection10m,WindSpeed10m,Zenith,AlbedoDaily,ptot,export_kwh,copper_ptot,Power Loss Event
tstamp,,,,,,,,,,,,,,,,,,,
2023-06-01 06:35:00,14.416667,-67.500127,18.666667,13.216667,2.166667,10.833333,1.000000,3.166667,22.816667,92.750000,943.166667,-115.666704,0.883333,93.001016,NaN,0.000,0.000,0.0000,2
2023-06-01 06:40:00,14.433333,-67.000102,16.833333,13.233333,4.333333,21.666667,2.000000,6.333333,22.733333,92.800000,943.233333,-115.333363,0.866667,92.000813,NaN,0.000,0.000,0.0000,2
2023-06-01 06:45:00,14.450000,-66.500000,15.000000,13.250000,6.500000,32.500000,3.000000,9.500000,22.650000,92.850000,943.300000,-115.000000,0.850000,91.000000,NaN,0.000,0.000,0.0000,2
2023-06-01 06:50:00,14.466667,-65.999898,13.166667,13.266667,8.666667,43.333333,4.000000,12.666667,22.566667,92.900000,943.366667,-114.666637,0.833333,89.999187,NaN,0.000,0.000,0.0000,2
2023-06-01 06:55:00,14.483333,-65.499873,11.333333,13.283333,10.833333,54.166667,5.000000,15.833333,22.483333,92.950000,943.433333,-114.333296,0.816667,88.998984,NaN,0.000,0.000,0.0000,2
2023-06-01 07:00:00,14.500000,-65.000000,9.500000,13.300000,13.000000,65.000000,6.000000,19.000000,22.400000,93.000000,943.500000,-114.000000,0.800000,88.000000,0.15,0.000,0.000,0.0000,2
2023-06-01 07:05:00,14.583333,-64.500127,8.816667,13.350000,19.166667,98.000000,11.166667,30.333333,22.350000,92.750000,943.566667,-113.833338,0.800000,87.001016,NaN,0.000,0.000,0.0000,2
2023-06-01 07:10:00,14.666667,-64.000102,8.133333,13.400000,25.333333,131.000000,16.333333,41.666667,22.300000,92.500000,943.633333,-113.666670,0.800000,86.000813,NaN,0.000,0.000,0.0000,2
2023-06-01 07:15:00,14.750000,-63.500000,7.450000,13.450000,31.500000,164.000000,21.500000,53.000000,22.250000,92.250000,943.700000,-113.500000,0.800000,85.000000,NaN,2.142,2.224,2.6777,0


In [24]:
# Define the start and end timestamps for the range
start_timestamp = pd.Timestamp('2024-03-20 08:30:00')
end_timestamp = pd.Timestamp('2024-03-20 19:00:00')

# Filter rows based on the timestamp range
filtered_df_ = filtered_df[(filtered_df.index >= start_timestamp) & (filtered_df.index < end_timestamp)]

filtered_df_.head(60)

,AirTemp,Azimuth,CloudOpacity,DewpointTemp,Dhi,Dni,Ebh,Ghi,PrecipitableWater,RelativeHumidity,SurfacePressure,WindDirection10m,WindSpeed10m,Zenith,AlbedoDaily,ptot,export_kwh,copper_ptot,Power Loss Event
tstamp,,,,,,,,,,,,,,,,,,,
2024-03-20 08:30:00,25.200000,-77.000000,0.000000,21.200000,109.000000,668.000000,330.000000,439.000000,37.200000,78.500000,945.000000,100.000000,1.300000,60.000000,0.15,21.441,21.64,25.8817,0
2024-03-20 08:35:00,25.283333,-76.167254,0.000000,21.216667,111.000000,677.000000,346.166667,457.166667,37.233333,78.200000,945.050000,100.000000,1.350000,59.001016,NaN,22.264,22.40,26.3362,0
2024-03-20 08:40:00,25.366667,-75.333804,0.000000,21.233333,113.000000,686.000000,362.333333,475.333333,37.266667,77.900000,945.100000,100.000000,1.400000,58.000813,NaN,23.049,23.10,27.8127,0
2024-03-20 08:45:00,25.450000,-74.500000,0.000000,21.250000,115.000000,695.000000,378.500000,493.500000,37.300000,77.600000,945.150000,100.000000,1.450000,57.000000,NaN,23.670,23.74,28.4880,0
2024-03-20 08:50:00,25.533333,-73.666196,0.000000,21.266667,117.000000,704.000000,394.666667,511.666667,37.333333,77.300000,945.200000,100.000000,1.500000,55.999187,NaN,24.371,24.45,29.3400,0
2024-03-20 08:55:00,25.616667,-72.832746,0.000000,21.283333,119.000000,713.000000,410.833333,529.833333,37.366667,77.000000,945.250000,100.000000,1.550000,54.998984,NaN,25.021,25.22,30.1635,0
2024-03-20 09:00:00,25.700000,-72.000000,0.000000,21.300000,121.000000,722.000000,427.000000,548.000000,37.400000,76.700000,945.300000,100.000000,1.600000,54.000000,0.15,25.851,25.92,31.1040,0
2024-03-20 09:05:00,25.866667,-71.167254,0.000000,21.333333,122.666667,728.166667,441.666667,564.333333,37.416667,76.066667,945.316667,100.166662,1.650000,52.834947,NaN,26.502,26.56,31.9786,0
2024-03-20 09:10:00,26.033333,-70.333804,0.000000,21.366667,124.333333,734.333333,456.333333,580.666667,37.433333,75.433333,945.333333,100.333330,1.700000,51.667958,NaN,27.059,27.20,32.5316,0


## Month, Week, Day Analysis

In [25]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

# Define features and target
features = ['AirTemp', 'Azimuth', 'CloudOpacity', 'DewpointTemp', 'Dhi', 'Dni', 'Ebh', 'Ghi', 'PrecipitableWater', 'RelativeHumidity', 'WindDirection10m', 'WindSpeed10m', 'Zenith']
target = 'ptot'

# Split the data into its labelled periods
filtered_df_normal = filtered_df[filtered_df['Power Loss Event'] == 0]
filtered_df_loadshedding = filtered_df[filtered_df['Power Loss Event'] == 1]
filtered_df_power_loss = filtered_df[filtered_df['Power Loss Event'] == 2]
filtered_df_data_gap = filtered_df[filtered_df['Power Loss Event'] == 3]

# Train a model on the normal periods
X_train = filtered_df_normal[features]
y_train = filtered_df_normal[target]

# Using a Random Forest Regressor for prediction
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
mse_cv_mean = -np.mean(cv_scores)  # Take the negative of the mean to get MSE
print(f'Cross-validated Mean Squared Error: {mse_cv_mean}')

# Train the final model on the full training set
model.fit(X_train, y_train)

# Predict ptot for LOADSHEDDING periods
X_loadshedding = filtered_df_loadshedding[features]
y_pred = model.predict(X_loadshedding)

# Predict ptot for POWER LOSS periods
X_power_loss = filtered_df_power_loss[features]
y_pred_pl = model.predict(X_power_loss)

# Predict ptot for DATA GAP periods
X_data_gap = filtered_df_data_gap[features]
y_pred_dg = model.predict(X_data_gap)

# Create new dfs with the estimated ptot
filtered_df_loadshedding = filtered_df_loadshedding.copy()
filtered_df_power_loss = filtered_df_power_loss.copy()
filtered_df_data_gap = filtered_df_data_gap.copy()

filtered_df_loadshedding['estimated_ptot'] = y_pred
filtered_df_power_loss['estimated_ptot'] = y_pred_pl
filtered_df_data_gap['estimated_ptot'] = y_pred_dg

# Combine the original ptot with the estimated values
filtered_df_combined = filtered_df.copy()
filtered_df_combined['predicted_ptot'] = filtered_df_combined['ptot']

filtered_df_combined.loc[filtered_df_combined['Power Loss Event'] == 1, 'predicted_ptot'] = filtered_df_loadshedding['estimated_ptot']
filtered_df_combined.loc[filtered_df_combined['Power Loss Event'] == 2, 'predicted_ptot'] = filtered_df_power_loss['estimated_ptot']
filtered_df_combined.loc[filtered_df_combined['Power Loss Event'] == 3, 'predicted_ptot'] = filtered_df_data_gap['estimated_ptot']

# Optional: Validate the model performance using some metrics
#X_test = filtered_df_normal[features]
#y_test = filtered_df_normal[target]
#y_test_pred = model.predict(X_test)
#mse = mean_squared_error(y_test, y_test_pred)
#print(f'Mean Squared Error on Test Data: {mse}')


Cross-validated Mean Squared Error: 22.081527293032273


In [26]:
filtered_df_combined.head(60)

,AirTemp,Azimuth,CloudOpacity,DewpointTemp,Dhi,Dni,Ebh,Ghi,PrecipitableWater,RelativeHumidity,SurfacePressure,WindDirection10m,WindSpeed10m,Zenith,AlbedoDaily,ptot,export_kwh,copper_ptot,Power Loss Event,predicted_ptot
tstamp,,,,,,,,,,,,,,,,,,,,
2023-06-01 06:35:00,14.416667,-67.500127,18.666667,13.216667,2.166667,10.833333,1.000000,3.166667,22.816667,92.750000,943.166667,-115.666704,0.883333,93.001016,NaN,0.000,0.000,0.0000,2,0.09074
2023-06-01 06:40:00,14.433333,-67.000102,16.833333,13.233333,4.333333,21.666667,2.000000,6.333333,22.733333,92.800000,943.233333,-115.333363,0.866667,92.000813,NaN,0.000,0.000,0.0000,2,0.29298
2023-06-01 06:45:00,14.450000,-66.500000,15.000000,13.250000,6.500000,32.500000,3.000000,9.500000,22.650000,92.850000,943.300000,-115.000000,0.850000,91.000000,NaN,0.000,0.000,0.0000,2,0.57262
2023-06-01 06:50:00,14.466667,-65.999898,13.166667,13.266667,8.666667,43.333333,4.000000,12.666667,22.566667,92.900000,943.366667,-114.666637,0.833333,89.999187,NaN,0.000,0.000,0.0000,2,0.89904
2023-06-01 06:55:00,14.483333,-65.499873,11.333333,13.283333,10.833333,54.166667,5.000000,15.833333,22.483333,92.950000,943.433333,-114.333296,0.816667,88.998984,NaN,0.000,0.000,0.0000,2,1.23660
2023-06-01 07:00:00,14.500000,-65.000000,9.500000,13.300000,13.000000,65.000000,6.000000,19.000000,22.400000,93.000000,943.500000,-114.000000,0.800000,88.000000,0.15,0.000,0.000,0.0000,2,1.49106
2023-06-01 07:05:00,14.583333,-64.500127,8.816667,13.350000,19.166667,98.000000,11.166667,30.333333,22.350000,92.750000,943.566667,-113.833338,0.800000,87.001016,NaN,0.000,0.000,0.0000,2,1.78353
2023-06-01 07:10:00,14.666667,-64.000102,8.133333,13.400000,25.333333,131.000000,16.333333,41.666667,22.300000,92.500000,943.633333,-113.666670,0.800000,86.000813,NaN,0.000,0.000,0.0000,2,2.21983
2023-06-01 07:15:00,14.750000,-63.500000,7.450000,13.450000,31.500000,164.000000,21.500000,53.000000,22.250000,92.250000,943.700000,-113.500000,0.800000,85.000000,NaN,2.142,2.224,2.6777,0,2.14200


In [27]:
# Count the number of rows where 'Power Loss Event' is 3
count_power_loss_event_3 = len(filtered_df_combined[filtered_df_combined['Power Loss Event'] == 3])

# Produce a new DataFrame where the rows are only those where 'Power Loss Event' is 1
df_power_loss_event_3 = filtered_df_combined[filtered_df_combined['Power Loss Event'] == 3]

# Print the count
print(f"Number of rows where 'Power Loss Event' is 3: {count_power_loss_event_3}")

df_power_loss_event_3.head(60)

# Optionally, save the new DataFrame to a CSV file
#df_power_loss_event_1.to_csv('df_power_loss_event_3.csv', index=False)

Number of rows where 'Power Loss Event' is 3: 1199


,AirTemp,Azimuth,CloudOpacity,DewpointTemp,Dhi,Dni,Ebh,Ghi,PrecipitableWater,RelativeHumidity,SurfacePressure,WindDirection10m,WindSpeed10m,Zenith,AlbedoDaily,ptot,export_kwh,copper_ptot,Power Loss Event,predicted_ptot
tstamp,,,,,,,,,,,,,,,,,,,,
2023-06-04 13:50:00,32.233333,29.335262,0.000000,8.966667,90.000000,806.666667,503.333333,593.333333,19.500000,23.766667,939.733333,-37.666637,4.033333,51.000102,NaN,21.467,0.0,0.0,3,20.99714
2023-06-05 13:50:00,33.066667,29.335262,7.066667,9.633333,146.000000,655.333333,405.333333,551.000000,19.766667,23.733333,941.233333,-59.999187,1.766667,51.666907,NaN,20.614,0.0,0.0,3,20.58330
2023-06-06 13:50:00,18.933333,29.335262,58.333333,14.666667,233.000000,0.000000,0.000000,233.000000,25.066667,76.100000,949.166667,113.999187,2.066667,51.666907,NaN,8.668,0.0,0.0,3,9.23894
2023-06-07 13:50:00,26.300000,29.335262,0.000000,16.200000,102.000000,774.666667,480.666667,582.666667,20.433333,53.866667,945.600000,19.666907,2.000000,51.666907,NaN,21.536,0.0,0.0,3,21.26615
2023-06-08 13:50:00,30.066667,29.335262,0.000000,9.800000,77.666667,841.666667,521.666667,599.333333,14.833333,28.466667,942.333333,-37.000000,2.100000,52.000102,NaN,21.277,0.0,0.0,3,21.50083
2023-06-09 11:10:00,27.500000,-19.002747,4.466667,10.166667,153.333333,684.666667,457.000000,610.000000,15.500000,34.000000,942.900000,-69.333330,2.100000,48.333363,NaN,25.601,0.0,0.0,3,23.95332
2023-06-09 13:05:00,31.216667,16.496570,26.633333,7.600000,331.333333,214.833333,144.666667,475.166667,16.433333,22.933333,941.133333,-63.500127,2.316667,47.499873,NaN,24.924,0.0,0.0,3,20.99357
2023-06-10 13:50:00,22.300000,29.335262,40.900000,14.166667,331.333333,24.666667,14.666667,346.000000,17.366667,60.000000,946.000000,126.999898,3.166667,52.000102,NaN,21.273,0.0,0.0,3,20.43733
2023-06-11 13:50:00,23.400000,28.667958,0.000000,12.433333,81.333333,837.666667,517.000000,598.333333,13.133333,50.033333,945.700000,9.333363,1.600000,52.000102,NaN,21.527,0.0,0.0,3,22.67154


In [ ]:
import matplotlib.pyplot as plt

# Assuming the user input is a start date
start_date = input("Enter the start date (YYYY-MM-DD): ")

# Convert the start date to datetime
start_date = pd.to_datetime(start_date)

# Define the end date by adding 7 days
end_date = start_date + pd.Timedelta(days=2)

# Extract the data for the specified 7-day period from both dataframes
old_data_period = filtered_df.loc[start_date:end_date]
new_data_period = filtered_df_combined.loc[start_date:end_date]

# Plotting
plt.figure(figsize=(12, 4))
plt.plot(old_data_period.index, old_data_period['ptot'], label='Actual', color='blue')
plt.plot(new_data_period.index, new_data_period['predicted_ptot'], label='Predicted', linestyle='--', color='red')
plt.xlabel('Date')
plt.ylabel('predicted_ptot')
plt.title('Old vs New Data for 2-Day Period')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import pandas as pd

# Define the year and month for which you want to calculate the total ptot
year = 2024
month = 5

# Filter the dataframes for the specified year and month
filtered_df_filtered = filtered_df[(filtered_df.index.year == year) & (filtered_df.index.month == month)]
filtered_df_combined_filtered = filtered_df_combined[(filtered_df_combined.index.year == year) & (filtered_df_combined.index.month == month)]

# Calculate the total actual and predicted ptot for the filtered dataframes
total_ptot_filtered = filtered_df_filtered['ptot'].sum()
total_ptot_combined = filtered_df_combined_filtered['predicted_ptot'].sum()

# Calculate kWh lost to loadshedding, power loss, and data gaps
loadshedding_loss = filtered_df_combined_filtered[filtered_df_combined_filtered['Power Loss Event'] == 1]['predicted_ptot'].sum()
power_loss = filtered_df_combined_filtered[filtered_df_combined_filtered['Power Loss Event'] == 2]['predicted_ptot'].sum()
data_gap_loss = filtered_df_combined_filtered[filtered_df_combined_filtered['Power Loss Event'] == 3]['predicted_ptot'].sum()

# Total kWh lost is the sum of losses
total_lost = loadshedding_loss + power_loss

print(f"Total actual kWh produced in {year}-{month:02} from VCOM: {total_ptot_filtered}")
print(f"Predicted total kWh produced in {year}-{month:02} without loadshedding and power loss: {total_ptot_combined}")
print(f"Loadshedding kWh lost: {loadshedding_loss}")
print(f"Power loss kWh lost: {power_loss}")
print(f"Total kWh lost: {total_lost}")

print(f"Data gap kWh by Copper meter: {data_gap_loss}")



In [ ]:
# Data for visualization
labels = ['Loadshedding Loss', 'Power Loss']
losses = [loadshedding_loss, power_loss]

# Create a bar chart
plt.figure(figsize=(10, 6))
plt.bar(labels, losses, color=['blue', 'orange', 'green'])
plt.xlabel('Loss Type')
plt.ylabel('kWh Lost')
plt.title(f'kWh Losses in {year}-{month:02}')
plt.show()